# 一、解题思路说明

## 待解决的问题
1. 训练集样本数少，且各id下的图片量分布严重不平衡，接近一半的id只有1张图片，需要对训练集进行增强处理。
2. 图片经过特殊加密处理，大部分图片的细节接近丢失，颜色信息不可用，需要使用鲁棒性和表征能力更强的深度模型和损失函数确保类内相似度高，类间相似度低。
3. 数据集规模较小，需要抑制过拟合

## 整体思路/方案
1. 使用***gaussian_blur***, ***mirror***, ***rotation***, ***resize_blur***, ***random_erasing***增强训练集数据，使用训练集的总体均值和方差做normalization。
2. 使用更大的w***eight_decay***和***dropout ratio***抵抗过拟合。
3. 为了获得更好的行人特征，采用***se_resnet_101_ibn_a***作为主干网络，最后一个layer的stride设为1，以减少下采样带来的信息损失，考虑到比赛所用的图片经过特殊预处理，基本只保留了全局信息，局部特征较为模糊，分类层采用***2-branch-PCB***，并使用BN层替代全连接层，进一步保留特征信息。
3. 使用rerank，使用先验信息对gallery进行聚类，优化排序结果。

## 数据处理
1. 读取训练集的路径信息，联合标签信息生成训练集，并从训练集中分割出校验集用于模型评估，保存为txt文件，每一行为图片的绝对路径和对应标签，格式为：
```
abs_path_0 label_0
abs_path_1 label_1
abs_path_2 label_2
...
abs_path
```
2. 使用训练好的模型反馈数据，提取训练集特征，使用层级聚类清洗训练集，去除重复图片。
3. 在线训练时使用上述所示的各种增强方法。

In [ ]:
# 数据清洗代码使用示例
SOURCE_TXT="path to your source trainset.txt to be cleaned"
SUB_WORKING_DIR="your sub working dir where you save the model"
CLEANED_OUTPUT_PATH="path to save your new trainset.txt after datacleaning"
GPU=0

# Start data clean
cd tools/
python3 data_clean.py --source_txt $SOURCE_TXT --sub_working_dir $SUB_WORKING_DIR --output_path $CLEANED_OUTPUT_PATH --drop_min 0 --clean_min 50 --gpu $GPU 


## 模型训练（损失函数）
   1. 本算法的模型训练分为两个阶段，第一阶段训练只包含***CrosseEntrophy Loss***的模型。
   2. 待模型收敛之后，开始第二阶段训练。第二阶段模型使用一阶段模型作为预训练模型，使用***Triplte Loss***和***CrosseEntrophy Loss***共同训练模型。

In [ ]:
## CrosseEntrophy Loss
nn.CrossEntropyLoss()

## Triplete Loss
class TripletLoss(nn.Module):
    def __init__(self, margin=0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.ranking_loss = nn.MarginRankingLoss(margin=margin, reduce=False) \
            if margin != "soft_margin" else nn.SoftMarginLoss(reduce=False)
        self.softmax = nn.Softmax(dim=1)
        self.softmin = nn.Softmin(dim=1)

    def forward(self, inputs, targets, step):
        # P x K
        n = inputs.size(0)

        # (optional) used ft_norm only on finetune situaation
        # inputs = 30 * inputs / torch.norm(inputs, p=2, dim=1, keepdim=True)

        # Compute pairwise distance
        dist = torch.pow(inputs, 2).sum(dim=1, keepdim=True).expand(n, n)
        dist = dist + dist.t()
        dist.addmm_(1, -2, inputs, inputs.t())
        dist = dist.clamp(min=1e-12).sqrt()

        # Batch hard mining
        pos_mask = targets.expand(n, n).eq(targets.expand(n, n).t())
        neg_mask = targets.expand(n, n).ne(targets.expand(n, n).t())
        pos_dist = dist[pos_mask].contiguous().view(n, -1)
        neg_dist = dist[neg_mask].contiguous().view(n, -1)
        dist_ap = torch.max(pos_dist, 1, keepdim=True)[0].squeeze(1)
        dist_an = torch.min(neg_dist, 1, keepdim=True)[0].squeeze(1)

        y = dist_an.data.new().resize_as_(dist_an.data).fill_(1)
        active_mask = dist_an.data.new().resize_as_(dist_an.data).fill_(1)
        y.requires_grad = False
        active_mask.requires_grad = False

        # Compute loss and statistic for logging.
        loss_mat = self.ranking_loss(dist_an, dist_ap, y) \
            if self.margin != "soft_margin"\
            else self.ranking_loss(dist_an - dist_ap, y)
        loss = loss_mat.mean()
        pull_ratio = (dist_an.data > dist_ap.data).sum().\
            float() * 100. / y.size(0)
        active_triplet = active_mask[loss_mat > 0.001].sum()
        mean_dist_an = dist_an.mean()
        mean_dist_ap = dist_ap.mean()

        return loss, pull_ratio, active_triplet, mean_dist_an, mean_dist_ap

## 主要创新点
1. 使用数据清洗和针对性的增强方法缓解数据不平衡等问题。
2. 因地制宜的对主干网络进行改进，使其极大地保留了图像的特征信息，提升模型整体对图片中的弱信息的表征能力。
3. 使用两阶段训练策略，先令模型学习各个类中心的信息，再通过额外的三元损失压缩各个类的特征空间，使得类与类之间更具区分性。
4. 后处理的匹配阶段，通过高置信度的配对过滤掉低置信度匹配，通过聚类进行gallery的特征的优化，详细代码参考generate_result.py

# 二、项目运行环境
## 预训练模型及相关论文
本算法使用***se_resnet_101_ibn_a***作为主干网络，使用了IBN-Net[1]的预训练参数
* 预训练模型下载链接为： https://drive.google.com/drive/folders/1thS2B8UOSBi_cJX6zRy6YYRwz_nVFI_S ，下载文件为***se_resnet101_ibn_a.pth.tar  ***
* 原作者给出的代码仓库地址为：https://github.com/XingangPan/IBN-Net  

[1] Xinggang Pan, Ping Luo, Jianping Shi, Xiaoou Tang. "Two at Once: Enhancing Learning and Generalization Capacities via IBN-Net", ECCV2018

## 项目所需的工具包/框架
* python v3.6.5
* cuda v9.0
* pytorch v1.1.0
* tensorflow v1.8
* torchvision v0.2.1
* tensorboardX v1.6


## 项目运行的资源环境

* 4卡11G CeForce GTX 1080Ti

# 三、项目运行办法
## 代码托管地址
   本算法根据大会要求托管于github私有仓库，可查看代码获得更详细的相关信息。  
   地址为：https://github.com/RamsleyJunting/NAIC_ReID

## 项目的文件结构
 |-- NAIC_ReID  
 
    |-- params.json
    |-- train.py
    |-- evaluate
        |-- __init__.py
        |-- generate_result.py
        |-- naic_evaluate.py
        |-- re_ranking.py
    |-- input_pipeline
        |-- __init__.py
        |-- iamge_data_reader.py
        |-- lmdb_dataset.py
        |-- samplers.py
    |-- nets
        |-- __init__.py
        |-- layers.py
        |-- model_main.py
        |-- nets_factory.py
        |-- resnet.py
        |-- se_resnet.py
    |-- utils
        |-- __init__.py
        |-- lr_scheduler.py
        |-- model_utils.py
        |-- training_utils.py


## 运行步骤
### 构建训练/校验集文本文件
   本项目采用pytorch的Dataloader读取list文件***trainset.txt***, 文件中每一行包含了每张图片的绝对路径和标签，用空格分隔，若要使用校验集评估模型的性能，需要自行从训练集中分割出一定比例数据作为***queryset.txt***和***gallery_set.txt***，格式与***trainset.txt***保持一致，如下所示：
```
abs_path_0 label_0
abs_path_1 label_1
abs_path_2 label_2
...
abs_path_n label_n
```
### 模型训练
   使用以下命令开始模型训练，通过***SUB_WORKING_DIR***指定模型的工作目录路径，训练过程中的train_log和模型文件将会保存在这里。***params.json***为模型训练需要的各种必要参数，对一些关键参数简要说明如下：
   1. ***"asoftmax_params"***: margin_softmax参数；
   2. ***"batches_dir"***: 数据集路径；
   3. ***"evaluation_params"***: 在线校验参数；
   4. ***"tri_loss_params"***: triplet loss参数；
   5. ***"working_dir"***: 模型工作路径SUB_WORKING_DIR的父目录

In [ ]:
python3 train.py --config_path params.json --sub_working_dir SUB_WORKING_DIR


### 在线校验与模型预测
   本项目包含在线校验模块，在训练过程中，通过设定***params.json***中"evaluation_params"-"eval_start=k"以及"evaluation_params"-"step=n"指示模型从第k个epoch开始，每n个epoch运行一次在线校验，并保存在校验集上表现最好的模型，保存为：***YOUR_SUB_WORKING_DIR/model_best.pth***。另外，若要单独预测校验集，请使用以下命令，若设定rerank_search，将对模型的rerank参数进行网格搜索，并将结果保存在***YOUR_SUB_WORKING_DIR/eval_log.log***

In [ ]:
python3 evaluate/naic_evaluate.py --gpu 0 --sub_working_dir YOUR_SUB_WORKING_DIR --rerank_search

### 结果文件生成
   运行以下命令以生成结果文件，其中**sub_working_dir**为模型及参数所在的目录，***k1,k2,lambda***为rerank的参数，***suffix***为自定义后缀，用于区分不同的结果文件，运行完成之后，将在evaluate/下生成***submit_suffix.json***，即为最终结果。

In [ ]:
python3 evaluate/generate_result.py --gpu 0 --sub_working_dir YOUR_SUB_WORKING_DIR --k1 $rerank_k1 --k2 $rerank_k2 --lambda $rerank_lambda --suffix modelbest